In [1]:
pip install AutoRAG

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json

In [2]:
#파일 로드 
#wiki 파일경로 - 임시로 test wiki
input_file = "rag_data/wiki_test.jsonl"
output_file = "rag_data/wiki_test_processed_wiki.jsonl"
with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        doc = json.loads(line)
        content = doc.get('content', '').strip()
        if content:
            # UTF-8 저장을 보장
            outfile.write(json.dumps({"id": doc["id"], "text": content}, ensure_ascii=False) + '\n')

print(f"Processed wiki data saved to {output_file}")

Processed wiki data saved to rag_data/wiki_test_processed_wiki.jsonl


In [3]:
# 테스트 질문 데이터 준비
test_data = pd.read_csv('rag_data/test.csv')

# 질문 데이터 출력 파일 경로
output_questions_file = "questions_with_choices_and_plus.jsonl"

# `problems`, `question_plus` 포함하여 데이터 저장
with open(output_questions_file, 'w', encoding='utf-8') as outfile:
    for _, row in test_data.iterrows():
        try:
            # 문제에서 질문과 선택지 텍스트 추출
            problem_raw = row['problems']
            # 이스케이프 문자 정리
            problem_cleaned = problem_raw.replace("\\", "\\\\").replace("'", '"')
            problem_data = json.loads(problem_cleaned)  # JSON으로 파싱
            
            question = problem_data.get('question', '').strip()  # 질문 텍스트
            choices = problem_data.get('choices', [])  # 선택지 리스트
            paragraph = row['paragraph']
            question_plus = row['question_plus'] if not pd.isna(row['question_plus']) else None  # 추가 질문

            # JSONL 형식으로 저장
            if question and choices:
                json_entry = {
                    "id": row['id'],
                    "question": question,
                    "choices": choices,
                    "paragraph": paragraph,
                    "question_plus": question_plus  # 추가 질문 포함
                }
                outfile.write(json.dumps(json_entry, ensure_ascii=False) + '\n')
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError for row {row['id']}: {e}")
        except Exception as e:
            print(f"Unexpected error for row {row['id']}: {e}")

print(f"Questions with choices and question_plus saved to {output_questions_file}")


JSONDecodeError for row generation-for-nlp-1343: Expecting ',' delimiter: line 1 column 20 (char 19)
JSONDecodeError for row generation-for-nlp-636: Expecting ',' delimiter: line 1 column 26 (char 25)
JSONDecodeError for row generation-for-nlp-1025: Expecting ',' delimiter: line 1 column 84 (char 83)
JSONDecodeError for row generation-for-nlp-1731: Expecting ',' delimiter: line 1 column 16 (char 15)
JSONDecodeError for row generation-for-nlp-1140: Expecting ',' delimiter: line 1 column 156 (char 155)
JSONDecodeError for row generation-for-nlp-1061: Expecting ',' delimiter: line 1 column 129 (char 128)
JSONDecodeError for row generation-for-nlp-635: Expecting ',' delimiter: line 1 column 30 (char 29)
JSONDecodeError for row generation-for-nlp-1121: Expecting ',' delimiter: line 1 column 59 (char 58)
JSONDecodeError for row generation-for-nlp-575: Expecting ',' delimiter: line 1 column 39 (char 38)
JSONDecodeError for row generation-for-nlp-1797: Expecting ',' delimiter: line 1 column 40

## Auto RAG 설정